In [1]:
import sys
sys.path.append('..')
from src.utilities import *
from src.models.train_model import train_model
from src.models.param_opt import bayes_parameter_opt_lgb

## 3. Modelling

### 3.1 Load master table

In [2]:
master = pd.read_csv(os.path.join(processed_path, 'master.csv'))

### 3.2 Train / val / test split

In [3]:
x_train = master[(master.sales.isna()==False)].drop(columns = ['region', 'brand']).copy()
x_train = x_train[x_train.month >= '2020-06']
x_train['train'] =  (x_train.month <= '2021-06').astype(float)
x_train.drop(columns = 'month', inplace = True)

### 3.3 Bayesian parameter search

In [4]:
best_params = bayes_parameter_opt_lgb(X = x_train.drop(columns = ['sales', 'train']), y = x_train.sales,
                                      init_round=100, opt_round=30, n_folds=5, random_seed=6, n_estimators=10000, 
                                      learning_rate=0.01, save_path = '', metric = 'mae').max['params']
best_params

|   iter    |  target   | baggin... | featur... | lambda_l1 | lambda_l2 | max_depth | min_ch... | min_sp... | num_le... |
-------------------------------------------------------------------------------------------------------------------------
[200]	cv_agg's l1: 838.211 + 17.9859
[400]	cv_agg's l1: 710.25 + 11.2366
[600]	cv_agg's l1: 694.793 + 13.1712
[800]	cv_agg's l1: 690.047 + 13.4761
[1000]	cv_agg's l1: 688.466 + 14.1735
[1200]	cv_agg's l1: 686.951 + 14.8763
|  1        | -686.9    |  0.8227   |  0.8796   |  3.644    |  1.054    |  14.2     |  40.98    |  0.06491  |  27.44    |
[200]	cv_agg's l1: 865.361 + 15.7375
[400]	cv_agg's l1: 719.649 + 6.78319
[600]	cv_agg's l1: 686.944 + 6.32511
[800]	cv_agg's l1: 673.731 + 4.53382
[1000]	cv_agg's l1: 667.236 + 4.33442
[1200]	cv_agg's l1: 664.085 + 4.76951
[1400]	cv_agg's l1: 663.359 + 5.4028
[1600]	cv_agg's l1: 662.426 + 5.97506
[1800]	cv_agg's l1: 662.283 + 6.39696
|  2        | -662.1    |  0.9412   |  0.2973   |  1.28     |  0.07203  | 

[200]	cv_agg's l1: 883.333 + 16.1446
[400]	cv_agg's l1: 702.46 + 6.71395
[600]	cv_agg's l1: 672.951 + 7.22847
[800]	cv_agg's l1: 663.391 + 9.37811
[1000]	cv_agg's l1: 657.378 + 9.43358
[1200]	cv_agg's l1: 654.869 + 9.07655
[1400]	cv_agg's l1: 653.734 + 9.10642
[1600]	cv_agg's l1: 653.173 + 9.05524
|  18       | -653.1    |  0.9276   |  0.4946   |  2.459    |  2.388    |  15.43    |  9.392    |  0.02961  |  20.1     |
[200]	cv_agg's l1: 892.997 + 19.6912
[400]	cv_agg's l1: 741.989 + 9.28752
[600]	cv_agg's l1: 720.956 + 13.7679
[800]	cv_agg's l1: 713.585 + 13.8593
[1000]	cv_agg's l1: 707.811 + 13.4209
[1200]	cv_agg's l1: 704.797 + 12.9941
[1400]	cv_agg's l1: 703.506 + 13.2916
[1600]	cv_agg's l1: 702.369 + 12.9528
[1800]	cv_agg's l1: 701.954 + 12.8717
|  19       | -701.9    |  0.9055   |  0.681    |  4.323    |  2.808    |  8.666    |  42.31    |  0.07856  |  19.26    |
[200]	cv_agg's l1: 799.232 + 16.8509
[400]	cv_agg's l1: 656.717 + 8.1963
[600]	cv_agg's l1: 643.584 + 7.23157
[800]	cv_

[1000]	cv_agg's l1: 679.86 + 8.06785
[1200]	cv_agg's l1: 677.813 + 8.4911
|  35       | -677.2    |  0.8941   |  0.3415   |  2.549    |  0.3054   |  8.647    |  36.96    |  0.08245  |  32.7     |
[200]	cv_agg's l1: 886.18 + 20.0049
[400]	cv_agg's l1: 761.101 + 10.6483
[600]	cv_agg's l1: 734.613 + 11.6361
[800]	cv_agg's l1: 718.002 + 10.1916
[1000]	cv_agg's l1: 709.754 + 9.52117
[1200]	cv_agg's l1: 705.219 + 9.6636
[1400]	cv_agg's l1: 701.537 + 9.48254
[1600]	cv_agg's l1: 699.129 + 8.87495
[1800]	cv_agg's l1: 697.833 + 8.82667
[2000]	cv_agg's l1: 697.261 + 8.56581
|  36       | -697.2    |  0.8509   |  0.4684   |  3.787    |  1.603    |  5.577    |  44.87    |  0.06301  |  32.43    |
[200]	cv_agg's l1: 856.562 + 20.3558
[400]	cv_agg's l1: 730.752 + 11.4698
[600]	cv_agg's l1: 710.873 + 11.0987
[800]	cv_agg's l1: 703.209 + 12.5553
[1000]	cv_agg's l1: 697.81 + 13.9766
[1200]	cv_agg's l1: 694.471 + 14.3435
[1400]	cv_agg's l1: 692.729 + 13.724
[1600]	cv_agg's l1: 692.292 + 13.5136
[1800]	cv_

[400]	cv_agg's l1: 680.944 + 7.22097
[600]	cv_agg's l1: 666.93 + 7.13895
[800]	cv_agg's l1: 664.557 + 9.38556
[1000]	cv_agg's l1: 663.628 + 11.4745
|  53       | -663.5    |  0.9656   |  0.6016   |  4.373    |  2.185    |  15.91    |  33.65    |  0.03429  |  35.51    |
[200]	cv_agg's l1: 808.348 + 19.408
[400]	cv_agg's l1: 660.683 + 10.493
[600]	cv_agg's l1: 645.144 + 6.87109
[800]	cv_agg's l1: 641.254 + 6.84098
[1000]	cv_agg's l1: 639.473 + 8.1654
[1200]	cv_agg's l1: 639.284 + 9.30259
|  54       | -639.1    |  0.8889   |  0.7449   |  2.315    |  2.894    |  11.49    |  5.963    |  0.08971  |  30.61    |
[200]	cv_agg's l1: 822.991 + 18.4173
[400]	cv_agg's l1: 707.697 + 12.6267
[600]	cv_agg's l1: 691.805 + 15.8441
[800]	cv_agg's l1: 687.737 + 16.7957
[1000]	cv_agg's l1: 686.696 + 17.1138
[1200]	cv_agg's l1: 685.881 + 16.7242
|  55       | -685.5    |  0.9927   |  0.7253   |  0.9199   |  1.497    |  12.03    |  43.99    |  0.008983 |  40.27    |
[200]	cv_agg's l1: 883.027 + 21.3366
[400

[200]	cv_agg's l1: 819.313 + 17.4847
[400]	cv_agg's l1: 703.744 + 10.6238
[600]	cv_agg's l1: 689.195 + 14.6844
[800]	cv_agg's l1: 684.526 + 15.9902
[1000]	cv_agg's l1: 682.679 + 16.4638
[1200]	cv_agg's l1: 681.366 + 17.281
[1400]	cv_agg's l1: 681.034 + 17.9128
|  71       | -680.8    |  0.8502   |  0.6729   |  2.218    |  2.603    |  11.44    |  40.65    |  0.02349  |  40.12    |
[200]	cv_agg's l1: 781.25 + 18.5451
[400]	cv_agg's l1: 661.306 + 10.2293
[600]	cv_agg's l1: 649.401 + 11.0833
[800]	cv_agg's l1: 647.678 + 11.5533
|  72       | -647.5    |  0.9187   |  0.7351   |  0.5744   |  1.211    |  15.03    |  24.23    |  0.07518  |  41.21    |
[200]	cv_agg's l1: 861.534 + 24.0108
[400]	cv_agg's l1: 731.927 + 11.4452
[600]	cv_agg's l1: 710.273 + 9.96003
[800]	cv_agg's l1: 703.041 + 11.9568
[1000]	cv_agg's l1: 698.397 + 13.6534
[1200]	cv_agg's l1: 696.314 + 15.2627
[1400]	cv_agg's l1: 694.449 + 14.5661
[1600]	cv_agg's l1: 693.593 + 14.9574
|  73       | -693.3    |  0.9176   |  0.3444   

[200]	cv_agg's l1: 914.027 + 15.9904
[400]	cv_agg's l1: 728.13 + 10.4914
[600]	cv_agg's l1: 693.328 + 9.03999
[800]	cv_agg's l1: 683.334 + 9.25709
[1000]	cv_agg's l1: 677.394 + 8.43736
[1200]	cv_agg's l1: 674.969 + 8.19899
[1400]	cv_agg's l1: 673.22 + 8.03743
[1600]	cv_agg's l1: 671.914 + 8.00682
|  89       | -671.7    |  0.9897   |  0.7325   |  1.228    |  2.576    |  15.37    |  22.14    |  0.09261  |  15.9     |
[200]	cv_agg's l1: 812.509 + 16.9432
[400]	cv_agg's l1: 693.262 + 9.5562
[600]	cv_agg's l1: 677.767 + 11.1613
[800]	cv_agg's l1: 670.732 + 11.5152
[1000]	cv_agg's l1: 665.895 + 10.8864
[1200]	cv_agg's l1: 663.323 + 10.8814
[1400]	cv_agg's l1: 662.899 + 11.212
[1600]	cv_agg's l1: 662.615 + 11.5101
|  90       | -662.4    |  0.9087   |  0.3666   |  3.464    |  0.2814   |  8.206    |  27.26    |  0.01982  |  42.36    |
[200]	cv_agg's l1: 814.637 + 15.635
[400]	cv_agg's l1: 663.399 + 5.86153
[600]	cv_agg's l1: 642.881 + 4.18826
[800]	cv_agg's l1: 637.649 + 4.4537
[1000]	cv_agg'

|  107      | -631.0    |  1.0      |  0.9      |  5.0      |  3.0      |  18.0     |  10.1     |  0.001    |  41.85    |
[200]	cv_agg's l1: 772.465 + 17.7909
[400]	cv_agg's l1: 646.921 + 11.9356
[600]	cv_agg's l1: 636.369 + 10.9273
|  108      | -636.1    |  1.0      |  0.9      |  0.0      |  3.0      |  18.0     |  20.75    |  0.001    |  45.0     |
[200]	cv_agg's l1: 790.364 + 18.6985
[400]	cv_agg's l1: 652.885 + 11.5594
[600]	cv_agg's l1: 637.965 + 8.283
[800]	cv_agg's l1: 636.44 + 8.99992
|  109      | -636.3    |  1.0      |  0.9      |  5.0      |  3.0      |  18.0     |  5.0      |  0.001    |  36.1     |
[200]	cv_agg's l1: 769.495 + 16.7406
[400]	cv_agg's l1: 643.34 + 12.3766
[600]	cv_agg's l1: 631.172 + 9.69221
|  110      | -631.0    |  1.0      |  0.9      |  0.4316   |  3.0      |  18.0     |  9.38     |  0.001    |  45.0     |
[200]	cv_agg's l1: 1003.27 + 25.4914
[400]	cv_agg's l1: 833.003 + 15.0551
[600]	cv_agg's l1: 769.265 + 10.2748
[800]	cv_agg's l1: 741.982 + 8.9550

[200]	cv_agg's l1: 794.169 + 18.812
[400]	cv_agg's l1: 654.684 + 12.6159
[600]	cv_agg's l1: 639.138 + 8.84215
[800]	cv_agg's l1: 637.583 + 8.9381
|  128      | -637.3    |  0.8      |  0.9      |  0.0      |  3.0      |  18.0     |  5.0      |  0.1      |  34.51    |
[200]	cv_agg's l1: 769.424 + 16.7438
[400]	cv_agg's l1: 642.503 + 11.7285
[600]	cv_agg's l1: 630.212 + 9.06823
|  129      | -629.9    |  1.0      |  0.9      |  0.0      |  3.0      |  18.0     |  16.07    |  0.1      |  45.0     |
[200]	cv_agg's l1: 772.874 + 17.9006
[400]	cv_agg's l1: 652.283 + 10.4414
[600]	cv_agg's l1: 639.407 + 9.32851
[800]	cv_agg's l1: 637.325 + 9.49215
[1000]	cv_agg's l1: 636.892 + 10.5104
|  130      | -636.6    |  0.8      |  0.9      |  5.0      |  3.0      |  10.16    |  5.0      |  0.1      |  45.0     |


{'bagging_fraction': 1.0,
 'feature_fraction': 0.9,
 'lambda_l1': 0.0,
 'lambda_l2': 3.0,
 'max_depth': 18.0,
 'min_child_weight': 16.07367872096262,
 'min_split_gain': 0.1,
 'num_leaves': 45.0}

### 3.4 Model training

In [5]:
##### Define best parameters found earlier
best_params = {'metric': 'mae',
               'bagging_fraction': 1,
               'feature_fraction': 0.9,
               'lambda_l1': 0,
               'lambda_l2': 3,
               'max_depth': 18,
               'min_child_weight': 16.07367872096262,
               'min_split_gain': 0.1,
               'num_leaves': 45}

##### Train a quantile regression lgb at different alpha levels
quantile_alphas = [0.1, 0.5, 0.9]
lgb_quantiles = {}

for alpha in quantile_alphas:
    current_model = train_model(x_train, target_name = 'sales', 
                                model_type = 'lgb-quantile', 
                                quantile_alpha = alpha,
                                params = best_params, 
                                metric = 'rmse', 
                                split = 'in_sample',
                                save_path = os.path.join(models_path, 
                                                         'model2_quantile_'+ str(alpha)+'.pkl'))
    lgb_quantiles[alpha] = current_model
    
##### Visualize feature contributions
feature_contributions = pd.DataFrame({'feature': x_train.drop(columns = ['sales', 'train']).columns, 
                                      'gain': lgb_quantiles[0.5].feature_importance(importance_type = 'gain'),
                                      'split': lgb_quantiles[0.5].feature_importance(importance_type = 'split')
                                     }).sort_values('gain', ascending = False)

feature_contributions[feature_contributions.gain>0][:40]



# 0.1 - 976.803  1033
# 0.5 - 547.665  697.595
# 0.9 -   650.551  999.234

####################     training with  4077      ####################
####################     validating with  453      ####################
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.026524 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 42500
[LightGBM] [Info] Number of data points in the train set: 4077, number of used features: 654
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	training's l1: 1792.37	valid_1's l1: 1729.3
Training until validation scores don't improve for 20 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2]	training's l1: 1773.58	valid_1's l1: 1710.37
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3]	training's l1: 1754.47	valid_1's l1: 1690.81
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4]	training's l1: 1740.19	valid_1's l1: 1675.92
[5]	trai

[154]	training's l1: 1174.67	valid_1's l1: 1169.94
[155]	training's l1: 1174.37	valid_1's l1: 1169.6
[156]	training's l1: 1173.48	valid_1's l1: 1168.72
[157]	training's l1: 1173.39	valid_1's l1: 1168.68
[158]	training's l1: 1172.45	valid_1's l1: 1168.31
[159]	training's l1: 1172.25	valid_1's l1: 1168.23
[160]	training's l1: 1171.65	valid_1's l1: 1167.77
[161]	training's l1: 1170.92	valid_1's l1: 1167.29
[162]	training's l1: 1170.31	valid_1's l1: 1166.78
[163]	training's l1: 1170.09	valid_1's l1: 1166.59
[164]	training's l1: 1169.21	valid_1's l1: 1166.23
[165]	training's l1: 1169.01	valid_1's l1: 1166.08
[166]	training's l1: 1168.91	valid_1's l1: 1166.09
[167]	training's l1: 1168.11	valid_1's l1: 1165.39
[168]	training's l1: 1167.47	valid_1's l1: 1165.25
[169]	training's l1: 1167.22	valid_1's l1: 1165.13
[170]	training's l1: 1166.31	valid_1's l1: 1164.22
[171]	training's l1: 1166.26	valid_1's l1: 1164.2
[172]	training's l1: 1165.17	valid_1's l1: 1163.49
[173]	training's l1: 1164.61	vali

[328]	training's l1: 1103.84	valid_1's l1: 1117.62
[329]	training's l1: 1102.66	valid_1's l1: 1116.64
[330]	training's l1: 1102.54	valid_1's l1: 1116.63
[331]	training's l1: 1102.4	valid_1's l1: 1116.54
[332]	training's l1: 1102.11	valid_1's l1: 1116.53
[333]	training's l1: 1101.88	valid_1's l1: 1116.49
[334]	training's l1: 1101.83	valid_1's l1: 1116.49
[335]	training's l1: 1101.45	valid_1's l1: 1116.13
[336]	training's l1: 1101.41	valid_1's l1: 1116.12
[337]	training's l1: 1101.15	valid_1's l1: 1116.11
[338]	training's l1: 1101.06	valid_1's l1: 1116
[339]	training's l1: 1101.04	valid_1's l1: 1116.02
[340]	training's l1: 1100.98	valid_1's l1: 1115.97
[341]	training's l1: 1100.77	valid_1's l1: 1115.84
[342]	training's l1: 1100.63	valid_1's l1: 1115.82
[343]	training's l1: 1100.58	valid_1's l1: 1115.79
[344]	training's l1: 1100.55	valid_1's l1: 1115.78
[345]	training's l1: 1099.86	valid_1's l1: 1115.27
[346]	training's l1: 1099.8	valid_1's l1: 1115.24
[347]	training's l1: 1099.79	valid_1

[496]	training's l1: 1075.48	valid_1's l1: 1097.56
[497]	training's l1: 1074.69	valid_1's l1: 1096.98
[498]	training's l1: 1074.64	valid_1's l1: 1096.94
[499]	training's l1: 1074.55	valid_1's l1: 1096.83
[500]	training's l1: 1074.53	valid_1's l1: 1096.83
[501]	training's l1: 1074.41	valid_1's l1: 1096.84
[502]	training's l1: 1073.82	valid_1's l1: 1096.26
[503]	training's l1: 1073.8	valid_1's l1: 1096.26
[504]	training's l1: 1073.17	valid_1's l1: 1095.72
[505]	training's l1: 1072.98	valid_1's l1: 1095.62
[506]	training's l1: 1072.88	valid_1's l1: 1095.61
[507]	training's l1: 1072.85	valid_1's l1: 1095.62
[508]	training's l1: 1072.72	valid_1's l1: 1095.49
[509]	training's l1: 1072.71	valid_1's l1: 1095.5
[510]	training's l1: 1072.59	valid_1's l1: 1095.49
[511]	training's l1: 1072.59	valid_1's l1: 1095.49
[512]	training's l1: 1072.55	valid_1's l1: 1095.53
[513]	training's l1: 1072.54	valid_1's l1: 1095.52
[514]	training's l1: 1072.3	valid_1's l1: 1095.42
[515]	training's l1: 1072.27	valid

[662]	training's l1: 1059.17	valid_1's l1: 1086.16
[663]	training's l1: 1059.14	valid_1's l1: 1086.15
[664]	training's l1: 1059.07	valid_1's l1: 1086.11
[665]	training's l1: 1059.05	valid_1's l1: 1086.1
[666]	training's l1: 1059.01	valid_1's l1: 1086.08
[667]	training's l1: 1058.32	valid_1's l1: 1085.84
[668]	training's l1: 1058.29	valid_1's l1: 1085.82
[669]	training's l1: 1058.24	valid_1's l1: 1085.76
[670]	training's l1: 1058.03	valid_1's l1: 1085.59
[671]	training's l1: 1057.91	valid_1's l1: 1085.46
[672]	training's l1: 1057.77	valid_1's l1: 1085.46
[673]	training's l1: 1057.56	valid_1's l1: 1085.42
[674]	training's l1: 1057.51	valid_1's l1: 1085.36
[675]	training's l1: 1057.49	valid_1's l1: 1085.36
[676]	training's l1: 1057.45	valid_1's l1: 1085.34
[677]	training's l1: 1057.43	valid_1's l1: 1085.33
[678]	training's l1: 1057.4	valid_1's l1: 1085.33
[679]	training's l1: 1057.29	valid_1's l1: 1085.16
[680]	training's l1: 1057.28	valid_1's l1: 1085.15
[681]	training's l1: 1057.27	vali

...

[1758]	training's l1: 657.469	valid_1's l1: 1003.9
[1759]	training's l1: 657.463	valid_1's l1: 1003.89
[1760]	training's l1: 657.459	valid_1's l1: 1003.89
[1761]	training's l1: 657.455	valid_1's l1: 1003.89
[1762]	training's l1: 657.452	valid_1's l1: 1003.88
[1763]	training's l1: 657.452	valid_1's l1: 1003.9
[1764]	training's l1: 657.431	valid_1's l1: 1003.89
[1765]	training's l1: 657.422	valid_1's l1: 1003.89
[1766]	training's l1: 657.416	valid_1's l1: 1003.89
[1767]	training's l1: 657.378	valid_1's l1: 1003.84
[1768]	training's l1: 657.37	valid_1's l1: 1003.83
[1769]	training's l1: 657.335	valid_1's l1: 1003.78
[1770]	training's l1: 657.283	valid_1's l1: 1003.72
[1771]	training's l1: 657.222	valid_1's l1: 1003.67
[1772]	training's l1: 657.22	valid_1's l1: 1003.67
[1773]	training's l1: 657.205	valid_1's l1: 1003.66
[1774]	training's l1: 657.194	valid_1's l1: 1003.66
[1775]	training's l1: 657.182	valid_1's l1: 1003.65
[1776]	training's l1: 657.157	valid_1's l1: 1003.65
[1777]	training'

[1924]	training's l1: 654.869	valid_1's l1: 1002.25
[1925]	training's l1: 654.864	valid_1's l1: 1002.25
[1926]	training's l1: 654.856	valid_1's l1: 1002.24
[1927]	training's l1: 654.85	valid_1's l1: 1002.25
[1928]	training's l1: 654.841	valid_1's l1: 1002.25
[1929]	training's l1: 654.833	valid_1's l1: 1002.24
[1930]	training's l1: 654.817	valid_1's l1: 1002.23
[1931]	training's l1: 654.807	valid_1's l1: 1002.23
[1932]	training's l1: 654.804	valid_1's l1: 1002.23
[1933]	training's l1: 654.794	valid_1's l1: 1002.23
[1934]	training's l1: 654.784	valid_1's l1: 1002.22
[1935]	training's l1: 654.775	valid_1's l1: 1002.22
[1936]	training's l1: 654.764	valid_1's l1: 1002.21
[1937]	training's l1: 654.758	valid_1's l1: 1002.21
[1938]	training's l1: 654.749	valid_1's l1: 1002.2
[1939]	training's l1: 654.743	valid_1's l1: 1002.2
[1940]	training's l1: 654.683	valid_1's l1: 1002.13
[1941]	training's l1: 654.678	valid_1's l1: 1002.12
[1942]	training's l1: 654.667	valid_1's l1: 1002.12
[1943]	training

[2088]	training's l1: 652.713	valid_1's l1: 1000.71
[2089]	training's l1: 652.709	valid_1's l1: 1000.71
[2090]	training's l1: 652.706	valid_1's l1: 1000.71
[2091]	training's l1: 652.698	valid_1's l1: 1000.7
[2092]	training's l1: 652.695	valid_1's l1: 1000.7
[2093]	training's l1: 652.691	valid_1's l1: 1000.7
[2094]	training's l1: 652.631	valid_1's l1: 1000.64
[2095]	training's l1: 652.628	valid_1's l1: 1000.63
[2096]	training's l1: 652.631	valid_1's l1: 1000.63
[2097]	training's l1: 652.629	valid_1's l1: 1000.63
[2098]	training's l1: 652.627	valid_1's l1: 1000.63
[2099]	training's l1: 652.621	valid_1's l1: 1000.63
[2100]	training's l1: 652.617	valid_1's l1: 1000.63
[2101]	training's l1: 652.586	valid_1's l1: 1000.62
[2102]	training's l1: 652.566	valid_1's l1: 1000.59
[2103]	training's l1: 652.556	valid_1's l1: 1000.59
[2104]	training's l1: 652.552	valid_1's l1: 1000.58
[2105]	training's l1: 652.524	valid_1's l1: 1000.55
[2106]	training's l1: 652.52	valid_1's l1: 1000.55
[2107]	training'

[2259]	training's l1: 651.167	valid_1's l1: 999.472
[2260]	training's l1: 651.16	valid_1's l1: 999.471
[2261]	training's l1: 651.158	valid_1's l1: 999.47
[2262]	training's l1: 651.103	valid_1's l1: 999.406
[2263]	training's l1: 651.095	valid_1's l1: 999.399
[2264]	training's l1: 651.093	valid_1's l1: 999.398
[2265]	training's l1: 651.089	valid_1's l1: 999.396
[2266]	training's l1: 651.085	valid_1's l1: 999.395
[2267]	training's l1: 651.085	valid_1's l1: 999.395
[2268]	training's l1: 651.082	valid_1's l1: 999.394
[2269]	training's l1: 651.08	valid_1's l1: 999.394
[2270]	training's l1: 651.078	valid_1's l1: 999.392
[2271]	training's l1: 651.075	valid_1's l1: 999.391
[2272]	training's l1: 651.072	valid_1's l1: 999.389
[2273]	training's l1: 651.059	valid_1's l1: 999.384
[2274]	training's l1: 651.052	valid_1's l1: 999.381
[2275]	training's l1: 651.042	valid_1's l1: 999.382
[2276]	training's l1: 651.04	valid_1's l1: 999.381
[2277]	training's l1: 651.004	valid_1's l1: 999.328
[2278]	training'

,feature,gain,split
8,sales_univ_brand_1mo,1253.745132,40
0,month_indicator,875.710247,22
123,sales_univ_brand_cumstd,789.714871,13
592,productGP_region_brand_3mo,304.843697,6
128,sales_region_b3_6mo,298.498230,25
121,sales_univ_brand_cummax,287.991561,4
136,sales_region_b3_cummin,255.102439,30
134,sales_region_b3_cumsum,250.294533,12
182,region_n_intmedandgen_perperson,233.206618,49
135,sales_region_b3_cummax,227.222517,20


### 3.5 Prediction storage

In [6]:
submission = master[(master.sales.isna()) & (master.month >= '2020-07')].copy()
submission['sales'] = lgb_quantiles[0.5].predict(submission.drop(columns = ['month', 'region', 'brand', 'sales']))
submission['lower'] = lgb_quantiles[0.1].predict(submission.drop(columns = ['month', 'region', 'brand', 'sales']))
submission['upper'] = lgb_quantiles[0.9].predict(submission.drop(columns = ['month', 'region', 'brand', 'sales', 'lower']))
submission = submission[['month', 'region', 'brand', 'sales', 'lower', 'upper']]

submission.loc[submission.sales < 0, 'sales'] = 0
submission.loc[submission.lower < 0, 'lower'] = 0
submission.loc[submission.upper < 0, 'upper'] = 0

submission.loc[submission.lower > submission.sales, 'lower'] = submission.sales
submission.loc[submission.upper < submission.sales, 'upper'] = submission.sales

submission.head()

,month,region,brand,sales,lower,upper
2714,2020-07,region_151,brand_1,14.577780,0.0,14.577780
2715,2020-07,region_151,brand_2,14.577780,0.0,14.577780
2716,2020-07,region_152,brand_1,18.293383,0.0,829.243822
2717,2020-07,region_152,brand_2,18.293383,0.0,829.243822
2718,2020-07,region_153,brand_1,51.156290,0.0,494.796230


In [7]:
submission.to_csv(os.path.join(results_path, 'submission11_team46.csv'), index = False)

In [8]:
(submission['upper'] - submission['lower']).mean()

# Submission 2 difference - 1815.437323083508
# Submission 3 difference - 1613.6602449420075
# Submission 4 difference - 1145.0976140382802
# Submission 5 difference - 1132.1785576554485
# 1222.15

1832.1783473827034

In [9]:
preds_train = pd.DataFrame({'month': master[(master.sales.isna()==False) & (master.month >= '2020-07')].month,
                            'region': master[(master.sales.isna()==False) & (master.month >= '2020-07')].region,
                            'brand': master[(master.sales.isna()==False) & (master.month >= '2020-07')].brand,
                            'predictions': lgb_quantiles[0.5].predict(master[(master.sales.isna()==False) &
                                                                             (master.month >= '2020-07')].drop(columns = ['month', 'region', 'brand', 'sales'])),
                            'lower': lgb_quantiles[0.1].predict(master[(master.sales.isna()==False) &
                                                                             (master.month >= '2020-07')].drop(columns = ['month', 'region', 'brand', 'sales'])),
                            'upper': lgb_quantiles[0.9].predict(master[(master.sales.isna()==False) &
                                                                             (master.month >= '2020-07')].drop(columns = ['month', 'region', 'brand', 'sales'])),
                            'sales': master[(master.sales.isna()==False) & (master.month >= '2020-07')].sales
                           })

preds_train.loc[preds_train.predictions < 0, 'predictions'] = 0
preds_train.loc[preds_train.lower < 0, 'lower'] = 0
preds_train.loc[preds_train.upper < 0, 'upper'] = 0

width = abs(preds_train['upper'] - preds_train['lower'])
below = (preds_train['lower'] - preds_train['sales'])*(preds_train['lower'] > preds_train['sales'])
above = (preds_train['sales'] - preds_train['upper'])*(preds_train['upper'] < preds_train['sales'])
outside = (2/0.2)*(below + above)

In [10]:
preds_train

,month,region,brand,predictions,lower,upper,sales
2412,2020-07,region_0,brand_1,31.791915,0.000000,164.680759,0.00
2413,2020-07,region_0,brand_2,31.791915,0.000000,164.680759,0.00
2414,2020-07,region_1,brand_1,135.803402,0.000000,499.677776,0.00
2415,2020-07,region_1,brand_2,135.803402,0.000000,499.677776,143.74
2416,2020-07,region_2,brand_1,27.882901,0.000000,77.049651,0.00
...,...,...,...,...,...,...,...
7935,2021-08,region_148,brand_2,5690.742390,1573.813731,5635.445018,4777.64
7936,2021-08,region_149,brand_1,2598.026795,1355.877009,3956.231328,2762.08
7937,2021-08,region_149,brand_2,721.880225,677.627469,1140.014869,998.29
7938,2021-08,region_150,brand_1,5749.099298,3228.418285,6621.892190,5634.64


In [11]:
preds_train['outside'] = (preds_train.sales < preds_train.lower)  | (preds_train.sales > preds_train.upper)
preds_train.outside.mean()

0.1620151371807001

In [11]:
width.mean()
# 0.25 988.73
# 0.2  1090.40
# 0.1 1827.48

1771.7814207767412

In [12]:
outside.mean()
# 0.25 1276.75
# 0.2  632.81
# 0.1 209.00

237.28347669251508

In [13]:
(width + outside).mean()
# 0.25 2265.492
# 0.2  1723.220
# 0.1 2036.48

2009.0648974692563